In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"
%matplotlib widget

In [ ]:
import addict
import colorcet as cc
import scipy.io as sio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.cm as cm
from matplotlib import collections
from matplotlib.colors import Normalize
from importlib import reload
from typing import List, Dict, Tuple
import celeri

plt.rcParams["text.usetex"] = False

# Read input, create data structures, close blocks


In [ ]:
COMMAND_FILE_NAME = "../data/config/anatolia_config.json"

model = celeri.build_model(COMMAND_FILE_NAME)

In [ ]:
celeri.plot_input_summary(model)

# Estimate block model parameters (dense)


In [ ]:
operators, estimation = celeri.assemble_and_solve_dense(model, tde=True, eigen=False)

# Plot model summary


In [ ]:
celeri.plot_estimation_summary(model, estimation)

# Standard plots

- Note that there are very few magic numbers here. The reason for this is that these values apply to almost all of the plots that follow so that by defining them up at the top we can make a single change and all figures are updated.


In [ ]:
WORLD_BOUNDARIES = sio.loadmat("WorldHiVectors.mat")
figsize_vectors = (15, 8)
fontsize = 16
lon_range = (25, 42)
lat_range = (34, 43)
lon_ticks = [25, 27, 29, 31, 33, 35, 37, 39, 41]
lat_ticks = [34, 36, 38, 40, 42]
slip_rate_min = -25.0
slip_rate_max = 25.0
land_color = "lightgray"
land_linewidth = 0.5
land_zorder = 0
key_rectangle_anchor = [39, 34]
key_rectangle_width = 3.0
key_rectangle_height = 1.55
key_arrow_lon = 40.5
key_arrow_lat = 34.5
key_arrow_magnitude = 25.0
key_arrow_text = "25 mm/yr"
key_arrow_color = "k"
key_background_color = "white"
key_linewidth = 1.0
key_edgecolor = "k"
arrow_magnitude_min = 0.0
arrow_magnitude_max = 25.0
arrow_colormap = cm.plasma
arrow_scale_default = 100
arrow_width = 0.0025
arrow_linewidth = 0.5
arrow_edgecolor = "k"
segment_line_width_outer = 2.0
segment_line_width_inner = 1.0
segment_line_color_outer = "k"
segment_line_color_inner = "w"

In [ ]:
def common_plot_elements(segment: pd.DataFrame, lon_range: Tuple, lat_range: Tuple):
    """Elements common to all subplots
    Args:
        segment (pd.DataFrame): Fault segments
        lon_range (Tuple): Longitude range (min, max)
        lat_range (Tuple): Latitude range (min, max)
    """
    for i in range(len(segment)):
        plt.plot(
            [segment.lon1[i], segment.lon2[i]],
            [segment.lat1[i], segment.lat2[i]],
            "-k",
            linewidth=segment_line_width_outer,
        )
    for i in range(len(segment)):
        plt.plot(
            [segment.lon1[i], segment.lon2[i]],
            [segment.lat1[i], segment.lat2[i]],
            "-w",
            linewidth=segment_line_width_inner,
        )

    plt.xlim([lon_range[0], lon_range[1]])
    plt.ylim([lat_range[0], lat_range[1]])
    plt.xticks(lon_ticks)
    plt.yticks(lat_ticks)
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlabel("longitude (degrees)", fontsize=fontsize)
    plt.ylabel("latitude (degrees)", fontsize=fontsize)
    plt.tick_params(labelsize=fontsize)


def plot_vel_arrows_elements(station, east_velocity, north_velocity, arrow_scale):
    # Draw velocity vectors
    velocity_magnitude = np.sqrt(east_velocity**2.0 + north_velocity**2.0)
    norm = Normalize()
    norm.autoscale(velocity_magnitude)
    norm.vmin = arrow_magnitude_min
    norm.vmax = arrow_magnitude_max
    colormap = arrow_colormap
    quiver_handle = plt.quiver(
        station.lon,
        station.lat,
        east_velocity,
        north_velocity,
        scale=arrow_scale_default * arrow_scale,
        width=arrow_width,
        scale_units="inches",
        color=colormap(norm(velocity_magnitude)),
        linewidth=arrow_linewidth,
        edgecolor=arrow_edgecolor,
    )

    # Draw land
    plt.fill(
        WORLD_BOUNDARIES["lon"],
        WORLD_BOUNDARIES["lat"],
        color=land_color,
        linewidth=land_linewidth,
        zorder=land_zorder,
    )

    # Draw white background rectangle
    rect = mpatches.Rectangle(
        key_rectangle_anchor,
        key_rectangle_width,
        key_rectangle_height,
        fill=True,
        color=key_background_color,
        linewidth=key_linewidth,
        ec=key_edgecolor,
    )
    plt.gca().add_patch(rect)

    # # Draw arrow legend
    plt.quiverkey(
        quiver_handle,
        key_arrow_lon,
        key_arrow_lat,
        key_arrow_magnitude,
        key_arrow_text,
        coordinates="data",
        color=key_arrow_color,
        fontproperties={"size": fontsize},
    )

    plt.gca().set_aspect("equal")
    plt.show()

In [ ]:
# Observed velocities
plt.figure(figsize=figsize_vectors)
plt.title("observed velocities", fontsize=fontsize)
common_plot_elements(model.segment, lon_range, lat_range)
plot_vel_arrows_elements(
    model.station,
    model.station.east_vel,
    model.station.north_vel,
    arrow_scale=1.0,
)

In [ ]:
# Model velocities
plt.figure(figsize=figsize_vectors)
plt.title("model velocities", fontsize=fontsize)
common_plot_elements(model.segment, lon_range, lat_range)
plot_vel_arrows_elements(
    model.station,
    estimation.east_vel,
    estimation.north_vel,
    arrow_scale=1.0,
)

In [ ]:
# Residual velocities
plt.figure(figsize=figsize_vectors)
plt.title("residual velocities", fontsize=fontsize)
common_plot_elements(model.segment, lon_range, lat_range)
plot_vel_arrows_elements(
    model.station,
    estimation.east_vel_residual,
    estimation.north_vel_residual,
    arrow_scale=0.25,
)

In [ ]:
# Rotation velocities
plt.figure(figsize=figsize_vectors)
plt.title("rotation velocities", fontsize=fontsize)
common_plot_elements(model.segment, lon_range, lat_range)
plot_vel_arrows_elements(
    model.station,
    estimation.east_vel_rotation,
    estimation.north_vel_rotation,
    arrow_scale=1.0,
)

In [ ]:
# Elastic segment velocities
plt.figure(figsize=figsize_vectors)
plt.title("elastic segment velocities", fontsize=fontsize)
common_plot_elements(model.segment, lon_range, lat_range)
plot_vel_arrows_elements(
    model.station,
    estimation.east_vel_elastic_segment,
    estimation.north_vel_elastic_segment,
    arrow_scale=0.25,
)

In [ ]:
# Elastic tde velocities
plt.figure(figsize=figsize_vectors)
plt.title("elastic mesh velocities", fontsize=fontsize)
common_plot_elements(model.segment, lon_range, lat_range)
plot_vel_arrows_elements(
    model.station,
    estimation.east_vel_tde,
    estimation.north_vel_tde,
    arrow_scale=0.25,
)

In [ ]:
# Residual velocities
residual_velocity_vector = np.concatenate(
    (estimation.east_vel_residual.values, estimation.north_vel_residual.values)
)
mean_average_error = np.mean(np.abs(residual_velocity_vector))

N_BINS = 100
BIN_EDGE_MIN = -15
BIN_EDGE_MAX = 15
BINS = np.linspace(BIN_EDGE_MIN, BIN_EDGE_MAX, N_BINS)

# Create histogram of residual velocities
plt.figure(figsize=(8, 4))
plt.hist(
    residual_velocity_vector,
    BINS,
    histtype="stepfilled",
    color="lightblue",
    edgecolor="k",
)
plt.xlim([BIN_EDGE_MIN, BIN_EDGE_MAX])
plt.xlabel("residual velocity (mm/yr)", fontsize=fontsize)
plt.ylabel("N", fontsize=fontsize)
plt.title(f"MAE = {mean_average_error:.2f} (mm/yr)", fontsize=fontsize)
plt.tick_params(labelsize=fontsize)
plt.show()

In [ ]:
# Scatter plot for velocity estimate errors
mae_station = np.abs(estimation.east_vel_residual.values) + np.abs(
    estimation.north_vel_residual.values
)
fig = plt.figure(figsize=figsize_vectors)
plt.title("residual velocities", fontsize=16)
plt.fill(
    WORLD_BOUNDARIES["lon"],
    WORLD_BOUNDARIES["lat"],
    color=land_color,
    linewidth=land_linewidth,
    zorder=land_zorder,
)

plt.scatter(
    model.station.lon,
    model.station.lat,
    s=25,
    edgecolors="k",
    c=mae_station,
    cmap="YlOrRd",
    linewidths=0.1,
)
common_plot_elements(model.segment, lon_range, lat_range)

# cbar = plt.colorbar(label="absolute error", shrink=0.45)
plt.clim(0, 10)

# Draw white background rectangle
rect = mpatches.Rectangle(
    key_rectangle_anchor,
    key_rectangle_width,
    key_rectangle_height,
    fill=True,
    color=key_background_color,
    linewidth=key_linewidth,
    ec=key_edgecolor,
)
plt.gca().add_patch(rect)
plt.show()

In [ ]:
# Plot estimated strike-slip rates
plt.figure(figsize=figsize_vectors)
plt.title("strike-slip rates", fontsize=fontsize)
common_plot_elements(model.segment, lon_range, lat_range)

plt.fill(
    WORLD_BOUNDARIES["lon"],
    WORLD_BOUNDARIES["lat"],
    color=land_color,
    linewidth=land_linewidth,
    zorder=land_zorder,
)

for i in range(len(model.segment)):
    if estimation.strike_slip_rates[i] < 0:
        plt.plot(
            [model.segment.lon1[i], model.segment.lon2[i]],
            [model.segment.lat1[i], model.segment.lat2[i]],
            "-r",
            linewidth=estimation.strike_slip_rates[i],
        )
    else:
        plt.plot(
            [model.segment.lon1[i], model.segment.lon2[i]],
            [model.segment.lat1[i], model.segment.lat2[i]],
            "-b",
            linewidth=estimation.strike_slip_rates[i],
        )

# Legend
black_segments = mlines.Line2D(
    [],
    [],
    color="red",
    marker="s",
    linestyle="None",
    markersize=10,
    label="right-lateral (10 mm/yr)",
)
red_segments = mlines.Line2D(
    [],
    [],
    color="blue",
    marker="s",
    linestyle="None",
    markersize=10,
    label="left-lateral (10 mm/yr)",
)
plt.legend(
    handles=[black_segments, red_segments],
    loc="lower right",
    fontsize=fontsize,
    framealpha=1.0,
    edgecolor="k",
).get_frame().set_boxstyle("Square")

In [ ]:
# Plot estimated strike-slip rates
plt.figure(figsize=figsize_vectors)
plt.title("dip-slip/tensile-slip rates", fontsize=fontsize)
common_plot_elements(model.segment, lon_range, lat_range)

plt.fill(
    WORLD_BOUNDARIES["lon"],
    WORLD_BOUNDARIES["lat"],
    color=land_color,
    linewidth=land_linewidth,
    zorder=land_zorder,
)

for i in range(len(model.segment)):
    if estimation.dip_slip_rates[i] > 0:
        plt.plot(
            [model.segment.lon1[i], model.segment.lon2[i]],
            [model.segment.lat1[i], model.segment.lat2[i]],
            "-r",
            linewidth=estimation.dip_slip_rates[i],
        )
    else:
        plt.plot(
            [model.segment.lon1[i], model.segment.lon2[i]],
            [model.segment.lat1[i], model.segment.lat2[i]],
            "-b",
            linewidth=estimation.dip_slip_rates[i],
        )

for i in range(len(model.segment)):
    if estimation.tensile_slip_rates[i] < 0:
        plt.plot(
            [model.segment.lon1[i], model.segment.lon2[i]],
            [model.segment.lat1[i], model.segment.lat2[i]],
            "-r",
            linewidth=estimation.tensile_slip_rates[i],
        )
    else:
        plt.plot(
            [model.segment.lon1[i], model.segment.lon2[i]],
            [model.segment.lat1[i], model.segment.lat2[i]],
            "-b",
            linewidth=estimation.tensile_slip_rates[i],
        )


# Legend
black_segments = mlines.Line2D(
    [],
    [],
    color="red",
    marker="s",
    linestyle="None",
    markersize=10,
    label="convergence (10 mm/yr)",
)
red_segments = mlines.Line2D(
    [],
    [],
    color="blue",
    marker="s",
    linestyle="None",
    markersize=10,
    label="extension (10 mm/yr)",
)
plt.legend(
    handles=[black_segments, red_segments],
    loc="lower right",
    fontsize=fontsize,
    framealpha=1.0,
    edgecolor="k",
).get_frame().set_boxstyle("Square")